In [ ]:
from ctgan import CTGAN
import pandas as pd
import os

# Đọc và xử lý dữ liệu
df = pd.read_csv(r"C:\Users\nguye\CODE\TimeSeries\BTL(Churn_Gan)\Data\WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.drop(columns=["customerID"], inplace=True)
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df_clean = df.dropna().reset_index(drop=True)
discrete_columns = df_clean.select_dtypes(include="object").columns.tolist()

In [6]:
# Tạo thư mục lưu
os.makedirs("data_parts", exist_ok=True)

In [7]:
# Lặp để tạo dữ liệu giả khác nhau
for i in range(1, 11):
    print(f"\n🔄 Đang huấn luyện mô hình CTGAN lần thứ {i}...")

    ctgan = CTGAN(epochs=300)
    ctgan.fit(df_clean.sample(frac=1, random_state=i).reset_index(drop=True), 
              discrete_columns=discrete_columns)
    
    print(f"🧬 Đang sinh dữ liệu part_{i}.csv ...")
    synth_data = ctgan.sample(1_000_000_000)  # 1 triệu dòng ~ 1GB nếu cột nhiều
    synth_data.to_csv(f"data_parts/part_{i}.csv", index=False)


🔄 Đang huấn luyện mô hình CTGAN lần thứ 1...
🧬 Đang sinh dữ liệu part_1.csv ...


RuntimeError: [enforce fail at alloc_cpu.cpp:116] data. DefaultCPUAllocator: not enough memory: you tried to allocate 158000 bytes.

In [ ]:
import os
import pandas as pd

csv_files = sorted([f for f in os.listdir("data_parts") if f.endswith(".csv")])
with open("TimeSeries_Data.csv", "w", encoding="utf-8", newline="") as f_out:
    first = True
    for file in csv_files:
        df = pd.read_csv(os.path.join("data_parts", file))
        df.to_csv(f_out, index=False, header=first, mode="a")
        first = False